# UD3 – Actividad 3.2: Control de calidad de ficheros CSV con firma

**Nombre y apellidos del alumno:**  
*(Completa este apartado antes de comenzar la actividad)*

## Introducción

La empresa **RetailCorp** opera en varios países europeos y gestiona su inventario de productos de forma distribuida.

Cada día, los distintos almacenes envían un *snapshot* del estado de su inventario en formato CSV a un sistema centralizado de analítica y planificación.

Durante un incidente reciente, se detectó que algunos ficheros contenían inconsistencias en su contenido y errores de integridad de archivo (hash incorrecto).

El área de **Data Engineering** ha decidido implementar un **control de calidad de entrada en el pipeline**, de forma que solo los ficheros correctos puedan avanzar hacia las etapas posteriores.

El objetivo de esta práctica es **diseñar e implementar un control de calidad de ficheros CSV con verificación de firma (SHA256/MD5)** de forma reproducible y automatizable.

## Dataset de trabajo

Se proporciona un fichero comprimido `RetailCorp-dataset-sample.tar.gz` que contiene una muestra de ficheros CSV de inventario junto con sus firmas.

### Contenido
- CSV: snapshots de inventario con campos `product_id`, `warehouse_id`, `snapshot_date`, `stock_units`, `reserved_units`.
- Firma: archivos `.sha256` o `.md5` correspondientes a cada CSV.

El alumno deberá extraer los ficheros y procesarlos siguiendo los pasos de la actividad.

# **Tareas a realizar**

### Tarea 1 – Preparación del entorno

Crear los directorios de trabajo (`incoming_raw`, `input`, `rejected`, `archive`) y extraer los ficheros del dataset proporcionado.
Se proporciona una función de preparación del entorno, no hay que modificar nada  

Aquí solo hay que analizar el código para familiarizarse con la estructura de directorios


In [1]:
import os
import subprocess

def prepare_environment(base_dir="data_pipeline",
                        dataset="RetailCorp-dataset-sample.tar.gz"):
    """
    Prepara la estructura de directorios y extrae el dataset inicial.
    """

    os.makedirs(f"{base_dir}/incoming_raw", exist_ok=True)
    os.makedirs(f"{base_dir}/input", exist_ok=True)
    os.makedirs(f"{base_dir}/rejected", exist_ok=True)
    os.makedirs(f"{base_dir}/archive", exist_ok=True)
    os.makedirs(f"{base_dir}/logs", exist_ok=True)

    subprocess.run(
        ["tar", "-xzf", dataset, "-C", f"{base_dir}/incoming_raw"],
        check=True
    )


def reset_data_pipeline(base_dir="data_pipeline"):
    """
    Elimina completamente el directorio de trabajo del pipeline
    para permitir una ejecución limpia del proceso.
    """
    if os.path.exists(base_dir):
        subprocess.run(
            ["rm", "-rf", base_dir],
            check=True
        )

reset_data_pipeline()
prepare_environment()

### Tarea 2 – Implementar la funcionalidad de gestión de la integridad en los ficheros de entrada

Se recomienda ir implementándolo en varias funciones que permitan dividir y estructurar el proceso.  
Puede realizarse en varias celdas.  

Registrar en un fichero de log la información de cada fichero procesado, indicando:
- Nombre de fichero
- Estado (`OK`, `REJECTED`, `ERROR`)
- Mensaje asociado

Mover los ficheros correctos al directorio `input`, los rechazados a `rejected` y archivar las firmas en `archive`.

In [2]:
import hashlib
import binascii

def calculate_hash(file_path, algorithm='sha256', chunk_size=4096):
    if algorithm == 'sha256':
        hash_func = hashlib.sha256()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                hash_func.update(chunk)
        return hash_func.hexdigest()

    elif algorithm == 'md5':
        hash_func = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                hash_func.update(chunk)
        return hash_func.hexdigest()

    else:
        raise ValueError("Algoritmo no soportado.")

# Función copiada del tutorial, mediante la cual calculamos el hash del fichero y detectar cambios o fallos. 

In [3]:
def read_signature(sig_path):

    with open(sig_path, 'r') as f:
        content = f.read().strip()
    # El hash es la primera palabra de la línea
    expected_hash = content.split()[0]
    
    if sig_path.endswith('.sha256'):
        algorithm = 'sha256'
    elif sig_path.endswith('.md5'):
        algorithm = 'md5'
    else:
        raise ValueError(f"Extensión de firma no reconocida: {sig_path}")
    
    return expected_hash, algorithm

# Función para leer el fichero asociado al csv y extraer el hash para comprobarlo

In [4]:
import datetime

LOG_FILE = "data_pipeline/logs/integrity_check.log"

def log_event(filename, status, message):

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"{timestamp} | {filename} | {status} | {message}\n"
    with open(LOG_FILE, 'a') as f:
        f.write(line)
    print(line.strip())

# Función para procesar el fichero y registrar el estado y usar un timestamp

In [5]:
import shutil
import glob

def process_files(base_dir="data_pipeline"):
    incoming = f"{base_dir}/incoming_raw"
    input_dir = f"{base_dir}/input"
    rejected_dir = f"{base_dir}/rejected"
    archive_dir = f"{base_dir}/archive"

    # Buscamos todos los ficheros CSV en incoming_raw (sin duplicados)
    csv_files = list(set(
        glob.glob(f"{incoming}/**/*.csv", recursive=True) +
        glob.glob(f"{incoming}/*.csv")
    ))

    if not csv_files:
        print("No se encontraron ficheros CSV en incoming_raw.")
        return

    for csv_path in csv_files:
        filename = os.path.basename(csv_path)

        sig_path = None
        for ext in ['.sha256', '.md5']:
            candidate = csv_path + ext
            if os.path.exists(candidate):
                sig_path = candidate
                break

        if sig_path is None:
            shutil.move(csv_path, os.path.join(rejected_dir, filename))
            log_event(filename, "REJECTED", "No se encontró fichero de firma asociado")
            continue

        try:
            expected_hash, algorithm = read_signature(sig_path)
            calculated_hash = calculate_hash(csv_path, algorithm=algorithm)
            sig_filename = os.path.basename(sig_path)

            if calculated_hash == expected_hash:
                shutil.move(csv_path, os.path.join(input_dir, filename))
                shutil.move(sig_path, os.path.join(archive_dir, sig_filename))
                log_event(filename, "OK", f"Integridad verificada ({algorithm}): {calculated_hash}")
            else:
                shutil.move(csv_path, os.path.join(rejected_dir, filename))
                shutil.move(sig_path, os.path.join(archive_dir, sig_filename))
                log_event(filename, "REJECTED",
                          f"Hash incorrecto ({algorithm}). Esperado: {expected_hash} | Calculado: {calculated_hash}")

        except Exception as e:
            log_event(filename, "ERROR", str(e))

In [6]:
process_files()

2026-02-18 16:48:36 | stock_snapshot_ES_WH_02_20260129.csv | REJECTED | Hash incorrecto (sha256). Esperado: 31b69a91ae0a42b48373b69279573b9d2612eb4dc9f7c142e1deb81f8e8d80ab | Calculado: 451aba4bcdcff581344f2bba735e703a94b15f63c7f5049deb3e36559afb2120
2026-02-18 16:48:36 | stock_snapshot_ES_WH_06_20260127.csv | REJECTED | Hash incorrecto (md5). Esperado: 854e5d63a7eaf0f7f11e10cb39b653cb | Calculado: 3570c6d4dd3028300334fa474fd6e209
2026-02-18 16:48:36 | stock_snapshot_FR_WH_02_20260129.csv | REJECTED | Hash incorrecto (sha256). Esperado: a7039cf19f7555f9f11b600adc18b7b2c215b7ecd89122db08df90dea845abcd | Calculado: 59271123818f76ead8b8d1ceec36f5825027003e9ac500d3654432dcc05a36f2
2026-02-18 16:48:36 | stock_snapshot_ES_WH_04_20260128.csv | OK | Integridad verificada (sha256): 3b8d3ed4e7c293e25ad0b8ab7fe1738c34f603530eec800b3624d869f62fa25f
2026-02-18 16:48:36 | stock_snapshot_ES_WH_01_20260129.csv | OK | Integridad verificada (sha256): 6a4f07384d1e2c459e9204b88462996fb3bc4d18be60d127d1b27

### Tarea 3 – Conteo de ficheros correctos e incorrectos

Mostrar el número total de ficheros procesados correctamente y los rechazados, para validar la actividad.  

Puede usarse cat y grep sobre el fichero de log generado por el proceso para obtener esta parte

In [7]:
import subprocess

result = subprocess.run(
    ["grep", "-c", "| OK |", LOG_FILE],
    capture_output=True, text=True
)
ok_count = int(result.stdout.strip()) if result.returncode == 0 else 0

result = subprocess.run(
    ["grep", "-c", "| REJECTED |", LOG_FILE],
    capture_output=True, text=True
)
rejected_count = int(result.stdout.strip()) if result.returncode == 0 else 0

print(f"✅ Ficheros aceptados (OK):       {ok_count}")
print(f"❌ Ficheros rechazados (REJECTED): {rejected_count}")
print(f"📦 Total procesados:              {ok_count + rejected_count}")

✅ Ficheros aceptados (OK):       5
❌ Ficheros rechazados (REJECTED): 4
📦 Total procesados:              9


reset_data_pipeline()
prepare_environment()

## Declaración de autoría y uso de Inteligencia Artificial Generativa (IA)

Durante la realización de esta actividad, declaro que (sustituye el cuadro por una X donde proceda):

☐ **No he utilizado herramientas de IA** durante la realización de esta actividad.  

☐ **He utilizado herramientas de IA como apoyo al aprendizaje**, para la consulta puntual de conceptos, aclaración de dudas o comprensión de la documentación, elaborando de forma autónoma el contenido entregado.  

☐ **Parte del contenido de la actividad ha sido generado con herramientas de IA**, siendo dicho contenido revisado, comprendido y adaptado críticamente antes de su entrega final.  

☐ **La mayor parte del contenido de la actividad ha sido generado con herramientas de IA**, existiendo una aportación propia limitada.  

La selección de esta opción implica asumir la responsabilidad sobre la autoría, comprensión y calidad del trabajo presentado.